In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# gpu support and model id
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"

# load pretrained model with gpu support
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
).to(device)

# needed for tokenizer and feature_extractor
processor = AutoProcessor.from_pretrained(model_id)

# transformers pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Supported Languages:

english, chinese, german, spanish, russian, korean, french, japanese, portuguese, turkish, polish, catalan, dutch, arabic, swedish, italian, indonesian, hindi, finnish, vietnamese, hebrew, ukrainian, greek, malay, czech, romanian, danish, hungarian, tamil, norwegian, thai, urdu, croatian, bulgarian, lithuanian, latin, maori, malayalam, welsh, slovak, telugu, persian, latvian, bengali, serbian, azerbaijani, slovenian, kannada, estonian, macedonian, breton, basque, icelandic, armenian, nepali, mongolian, bosnian, kazakh, albanian, swahili, galician, marathi, punjabi, sinhala, khmer, shona, yoruba, somali, afrikaans, occitan, georgian, belarusian, tajik, sindhi, gujarati, amharic, yiddish, lao, uzbek, faroese, haitian creole, pashto, turkmen, nynorsk, maltese, sanskrit, luxembourgish, myanmar, tibetan, tagalog, malagasy, assamese, tatar, hawaiian, lingala, hausa, bashkir, javanese, sundanese, cantonese, burmese, valencian, flemish, haitian, letzeburgesch, pushto, panjabi, moldavian, moldovan, sinhalese, castilian, mandarin

In [24]:
from IPython.display import Audio
from datasets import load_dataset

# load audio dataset
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

sample_audio_data = dataset[0]["audio"]["array"]

In [8]:
# translate to german

result = pipe(sample_audio_data, generate_kwargs={"language": "german"})
print(result["text"])

 Mr. Quilter ist der Apostel der mittleren Klasse und wir sind froh, dass wir sein Gospelswesen begrüßen. Nichts ist Mr. Quilters Art, weniger interessant als sein Wissen. Er erzählt uns, dass in dieser festlichen Saison des Jahres, mit Weihnachten und gebratenem Fleisch vor uns, Similes, die von dem Essen ausgesucht werden, am liebsten in der Meinung kommen. Er hat große Widersprüche, ob Sir Frederick Leighton's Arbeit wirklich griechisch ist, und kann in es nur wenig von Rocky Ithaka entdecken. Linnells Bilder sind eine Art von Upguards und Adam-Bildern, und Masons exquisite Idylls sind so national wie ein Jingo-Poem. Mr. Burkett Fosters Landschaft lacht an einem, so wie Mr. Carker seine Zähne anzeigen wollte. Und Mr. John Collier gibt seinem Sitter einen leckeren Schlag in die Rüste, bevor er sagt, wie ein Shampooer in einem türkischen Bad, Nächster Mann!


In [28]:
# standard is english

result = pipe(sample_audio_data)
print(result["text"])

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [23]:
# translate to french

result = pipe(sample_audio_data, generate_kwargs={"language": "french"})
print(result["text"])

 Mr. Quilter est l'apostol des classes de la middle class et nous sommes heureux de recevoir son gospel. Ni est-il qu'il n'est pas plus intéressant que son matière. Il nous dit qu'à cette saison festive du an, avec la Noël et le poivre de la pâte qui se déclarent devant nous, les symboles détaillés de l'alimentation et leurs résultats s'occuperont le plus rapidement de la tête. Il a des doutes graves de savoir si le travail de Sir Frederick Leighton est vraiment grec, et peut découvrir dans le travail de Sir Frederick Leighton, mais peu de Rocky Ithaca. Les images de Linnell sont un genre de peintures d'upguards et d'adams, et les idoles exquisites de Mason sont aussi nationales qu'une poème d'ingots. Les chansons sont aussi nationales qu'un poème d'ingots. Les landscapes de Mr. Burkitt Foster rient à l'un de l'autre, de la même manière que Mr. Carker flashait ses doigts. Et Mr. John Collier donne à son assiette un slap à la tête, avant de dire, comme un shampooer dans un bain de bain 

In [27]:
# translate to russian

result = pipe(sample_audio_data, generate_kwargs={"language": "russian"})
print(result["text"])

 Мистер Квилтер — апостол средневекового класса, и мы рады приветствовать его Библию. Ниже мистер Квилтера манера менее интересна, чем его тема. Он говорит нам, что в этом фестивальном сезоне года, с Кристосом и курицей, символы, выделенные из еды, и их результаты, оказываются самостоятельно в голове. Он в глубокими сомнениями, что его творчество действительно греческое, и не может найти в нем ни одного из «Рокки Итхака». Линеллские картинки — как будто бы «Адам и Аббат» и Мейсонские эксклюзивные «Иттл» как будто бы «Национальный поэм»жинго. Мистер Беркетт Фостер смеется на них, как и мистер Каркер, когда он шепчет свои зубы. И мистер Джон Коллиер дает сидору радостную слапку в задницу, пока он говорит, как шампунь и бутылка, «Новый человек!»
